In [2]:
import cv2
import os
import uuid
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer, Conv2D, Dense, MaxPooling2D, Input, Flatten
from tensorflow.keras.metrics import Precision, Recall

In [4]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

I0000 00:00:1727860846.496456   11147 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1727860846.518746   11147 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1727860846.522566   11147 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355


In [5]:
POS_PATH = os.path.join('data', 'positive')
NEG_PATH = os.path.join('data', 'negative')
ANCHOR_PATH = os.path.join('data', 'anchor')

In [6]:
os.makedirs(POS_PATH, exist_ok=True)
os.makedirs(NEG_PATH, exist_ok=True)
os.makedirs(ANCHOR_PATH, exist_ok=True)

In [7]:
for folder in os.listdir('lfw'):
    for file in os.listdir(os.path.join('lfw', folder)):
        OLD_PATH = os.path.join('lfw', folder, file)
        NEW_PATH = os.path.join(NEG_PATH, file)
        os.replace(OLD_PATH, NEW_PATH)

In [8]:
def crop_frame(frame, size=(250,250)):
    if frame.shape[0] < size[0] or frame.shape[1] < size[1]:
        return None
    x_rem = frame.shape[0] - size[0]
    y_rem = frame.shape[1] - size[1]
    cropped_frame = frame[x_rem//2:x_rem//2+size[0], y_rem//2:y_rem//2+size[1]]
    return cropped_frame

In [9]:
import uuid

In [10]:
cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()
    frame = crop_frame(frame)
    
    key = cv2.waitKey(1) & 0xFF
    
    if key == ord('p'):
        img_name = os.path.join(POS_PATH, str(uuid.uuid1()) + '.jpg')
        cv2.imwrite(img_name, frame)
    
    if key == ord('a'):
        img_name = os.path.join(ANCHOR_PATH, str(uuid.uuid1()) + '.jpg')
        cv2.imwrite(img_name, frame)
    
    cv2.imshow('Image Collection', frame)
    
    if key == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [11]:
input_image = tf.data.Dataset.list_files(os.path.join(ANCHOR_PATH, '*.jpg')).take(300)
val_image = tf.data.Dataset.list_files(os.path.join(POS_PATH, '*.jpg')).take(300)
negative = tf.data.Dataset.list_files(os.path.join(NEG_PATH, '*.jpg')).take(300)

I0000 00:00:1727860850.059924   11147 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1727860850.062717   11147 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1727860850.065267   11147 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1727860850.169440   11147 cuda_executor.cc:1015] successful NUMA node read from SysFS ha

In [12]:
def preprocess(file_path):
    img = tf.io.read_file(file_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, (100, 100))
    img = img / 255.0
    return img

In [13]:
positives = tf.data.Dataset.zip((input_image, val_image, tf.data.Dataset.from_tensor_slices(tf.ones(len(input_image)))))
negatives = tf.data.Dataset.zip((input_image, negative, tf.data.Dataset.from_tensor_slices(tf.zeros(len(input_image)))))
data = positives.concatenate(negatives)

In [14]:
def preprocess_twin(anchor, other, label):
    return preprocess(anchor), preprocess(other), label

In [15]:
sample = data.as_numpy_iterator().next()
preprocess_twin(*sample)

(<tf.Tensor: shape=(100, 100, 3), dtype=float32, numpy=
 array([[[0.7767157 , 0.8159314 , 0.8512255 ],
         [0.7683824 , 0.80759805, 0.84289217],
         [0.7644608 , 0.8007353 , 0.8301471 ],
         ...,
         [0.627451  , 0.6117647 , 0.67058825],
         [0.64460784, 0.62892157, 0.6877451 ],
         [0.6460784 , 0.63039213, 0.68921566]],
 
        [[0.782598  , 0.8218137 , 0.8571078 ],
         [0.77230394, 0.8125    , 0.8448529 ],
         [0.7647059 , 0.80196077, 0.82843137],
         ...,
         [0.61740196, 0.6017157 , 0.6610294 ],
         [0.62916666, 0.6134804 , 0.6742647 ],
         [0.65686274, 0.64117646, 0.7019608 ]],
 
        [[0.7862745 , 0.8254902 , 0.8607843 ],
         [0.7752451 , 0.81764704, 0.84338236],
         [0.7647059 , 0.80196077, 0.82843137],
         ...,
         [0.5884804 , 0.57279414, 0.6335784 ],
         [0.6272059 , 0.61151963, 0.6781863 ],
         [0.6460784 , 0.63039213, 0.6970588 ]],
 
        ...,
 
        [[0.61789215, 0.6404412 

In [16]:
data = data.map(preprocess_twin)
data = data.cache()
data = data.shuffle(buffer_size=1024)

In [17]:
train_data = data.take(round(0.7*len(data)))
train_data = train_data.batch(16)
train_data = train_data.prefetch(8)

In [18]:
test_data = data.skip(round(0.7*len(data)))
test_data = test_data.take(round(0.3*len(data)))
test_data = test_data.batch(16)
test_data = test_data.prefetch(8)

In [19]:
def make_embedding():
    inp = Input(shape=(100, 100, 3))
    c1 = Conv2D(64, (10, 10), activation='relu')(inp)
    m1 = MaxPooling2D((2, 2))(c1)
    c2 = Conv2D(128, (7, 7), padding='same')(m1)
    m2 = MaxPooling2D((2, 2))(c2)
    c3 = Conv2D(128, (4, 4), padding='same')(m2)
    m3 = MaxPooling2D((2, 2))(c3)
    c4 = Conv2D(256, (4, 4), padding='same')(m3)
    f = Flatten()(c4)
    d = Dense(4096, activation='sigmoid')(f)
    
    return Model(inputs=inp, outputs=d)

In [20]:
embedding = make_embedding()

In [21]:
embedding.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 100, 100, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 91, 91, 64)     │        19,264 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 45, 45, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 45, 45, 128)    │       401,536 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 22, 22, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 22, 22, 128)    │       262,272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 11, 11, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 11, 11, 256)    │       524,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 30976)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 4096)           │   126,881,792 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 128,089,408 (488.62 MB)

 Trainable params: 128,089,408 (488.62 MB)

 Non-trainable params: 0 (0.00 B)

In [22]:
class L1Distance(Layer):
    def __init__(self, **kwargs):
        super(L1Distance, self).__init__(**kwargs)
    
    def call(self, input_embeddings, validation_embeddings):
        return tf.math.abs(input_embeddings - validation_embeddings)

In [23]:
def make_siamese_model():
    input_image = Input(shape=(100, 100, 3))
    val_image = Input(shape=(100, 100, 3))
    
    siamese_layer = L1Distance()
    siamese_layer._name = 'distance'
    
    distances = siamese_layer(embedding(input_image), embedding(val_image))
    
    classifier = Dense(1, activation='sigmoid')(distances)
    
    return Model(inputs=[input_image, val_image], outputs=classifier, name='SiameseNetwork')

In [24]:
siamese_model = make_siamese_model()
siamese_model.summary()

Model: "SiameseNetwork"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 100, 100,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_2       │ (None, 100, 100,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ functional          │ (None, 4096)      │ 128,089,4… │ input_layer_1[0]… │
│ (Functional)        │                   │            │ input_layer_2[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ l1_distance         │ (None, 4096)      │          0 │ functional[0][0], │
│ (L1Distance)        │                   │            │ functional[1][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 1)         │      4,097 │ l1_distance[0][0] │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 128,093,505 (488.64 MB)

 Trainable params: 128,093,505 (488.64 MB)

 Non-trainable params: 0 (0.00 B)

In [25]:
binary_cross_loss = tf.keras.losses.BinaryCrossentropy()

In [26]:
opt = tf.keras.optimizers.Adam(1e-4)

In [27]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(opt=opt, siamese_model=siamese_model)

os.makedirs(checkpoint_dir, exist_ok=True)

In [28]:
@tf.function
def train_step(batch):
    
    with tf.GradientTape() as tape:
        
        X = batch[:2]
        y = batch[2]
        
        y_hat = siamese_model(X, training=True)
        loss = binary_cross_loss(y, y_hat)
        
    grad = tape.gradient(loss, siamese_model.trainable_variables)
    opt.apply_gradients(zip(grad, siamese_model.trainable_variables))
    
    return loss


In [29]:
def train(data, EPOCHS):
    for epoch in range(EPOCHS):
        print(f'Epoch {epoch+1}/{EPOCHS}')
        progbar = tf.keras.utils.Progbar(len(data))
        
        for idx, batch in enumerate(data):
            loss = train_step(batch)
            progbar.update(idx+1, values=[('loss', loss)])
            
        if (epoch+1) % 10 == 0:
            checkpoint.save(file_prefix=checkpoint_prefix)

In [30]:
EPOCHS = 10

In [31]:
train(train_data, EPOCHS)

Epoch 1/10


2024-10-02 14:50:52.002593: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8907
W0000 00:00:1727860852.055298   11245 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1727860852.073255   11243 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1727860852.082476   11245 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1727860852.083906   11243 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1727860852.086258   11245 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1727860852.092286   11243 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1727860852.094953   11245 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1727860852.098289   11243 gpu_t

 1/27 ━━━━━━━━━━━━━━━━━━━━ 1:12 3s/step - loss: 0.6931

W0000 00:00:1727860853.804901   11243 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1727860853.810816   11243 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1727860853.813498   11243 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1727860853.817245   11243 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1727860853.819732   11243 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1727860853.822474   11243 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1727860853.825993   11243 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1727860853.828671   11243 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1727860853.831344   11243 gp

26/27 ━━━━━━━━━━━━━━━━━━━━ 0s 162ms/step - loss: 0.2949

...

27/27 ━━━━━━━━━━━━━━━━━━━━ 4s 166ms/step - loss: 0.0261
Epoch 4/10
27/27 ━━━━━━━━━━━━━━━━━━━━ 4s 166ms/step - loss: 0.0069
Epoch 5/10
 1/27 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0020

2024-10-02 14:51:12.773426: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


27/27 ━━━━━━━━━━━━━━━━━━━━ 4s 166ms/step - loss: 0.0029
Epoch 6/10
27/27 ━━━━━━━━━━━━━━━━━━━━ 4s 166ms/step - loss: 0.0021
Epoch 7/10
27/27 ━━━━━━━━━━━━━━━━━━━━ 4s 166ms/step - loss: 0.0013
Epoch 8/10
27/27 ━━━━━━━━━━━━━━━━━━━━ 4s 166ms/step - loss: 8.3258e-04
Epoch 9/10
 1/27 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 9.9124e-04

2024-10-02 14:51:30.624019: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


27/27 ━━━━━━━━━━━━━━━━━━━━ 4s 166ms/step - loss: 4.1747e-04
Epoch 10/10
27/27 ━━━━━━━━━━━━━━━━━━━━ 4s 166ms/step - loss: 3.0442e-04


In [33]:
test_input, test_val, y_true = test_data.as_numpy_iterator().next()

In [34]:
y_hat = siamese_model.predict([test_input, test_val])
res = [1 if i > 0.5 else 0 for i in y_hat]

I0000 00:00:1727860901.306144   11242 service.cc:146] XLA service 0x7f2920001670 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1727860901.306169   11242 service.cc:154]   StreamExecutor device (0): NVIDIA GeForce RTX 3050 6GB Laptop GPU, Compute Capability 8.6
2024-10-02 14:51:41.311508: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 858ms/step


I0000 00:00:1727860902.101292   11242 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


In [35]:
res

[0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1]

In [36]:
y_true

array([0., 0., 0., 0., 1., 1., 0., 1., 1., 0., 0., 1., 1., 1., 0., 1.],
      dtype=float32)

In [37]:
recall = Recall()
precision = Precision()

recall.update_state(y_true, res)
precision.update_state(y_true, res)

In [38]:
recall.result().numpy(), precision.result().numpy()

(1.0, 1.0)

In [40]:
siamese_model.save('siamese_model.h5')

In [41]:
model = tf.keras.models.load_model('siamese_model.h5', custom_objects={'L1Distance': L1Distance, 'BinaryCrossentropy': binary_cross_loss})

In [42]:
model.predict([test_input, test_val])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 166ms/step


array([[1.1694588e-07],
       [1.5159608e-05],
       [1.6396921e-08],
       [2.5071617e-07],
       [9.9999750e-01],
       [9.9999380e-01],
       [5.9653407e-09],
       [9.9998581e-01],
       [9.9999356e-01],
       [4.6600184e-07],
       [1.3911546e-10],
       [1.0000000e+00],
       [9.9930286e-01],
       [1.0000000e+00],
       [8.5158223e-05],
       [9.9999666e-01]], dtype=float32)

In [43]:
model.summary()

Model: "SiameseNetwork"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 100, 100,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_2       │ (None, 100, 100,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ functional          │ (None, 4096)      │ 128,089,4… │ input_layer_1[0]… │
│ (Functional)        │                   │            │ input_layer_2[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ l1_distance         │ (None, 4096)      │          0 │ functional[0][0], │
│ (L1Distance)        │                   │            │ functional[1][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 1)         │      4,097 │ l1_distance[0][0] │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 128,093,505 (488.64 MB)

 Trainable params: 128,093,505 (488.64 MB)

 Non-trainable params: 0 (0.00 B)

In [46]:
def verify(model, detection_threshold, verification_threshold):
    results = []
    
    for image in os.listdir(os.path.join('application_data','verification_images')):
        input_image = preprocess(os.path.join('application_data','input_image', 'input_image.jpg'))
        validation_image = preprocess(os.path.join('application_data','verification_images', image))
        
        result = model.predict(list(np.expand_dims([input_image, validation_image], axis=1)))
        results.append(result)
        
    detection = np.sum(np.array(results) > detection_threshold)
    verification = detection/len(os.listdir(os.path.join('application_data','verification_images')))
    verified = verification > verification_threshold
    
    return results, verified   

In [51]:
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    frame = crop_frame(frame)
    
    cv2.imshow('Verification', frame)
    
    if cv2.waitKey(3) & 0xFF == ord('v'):
        cv2.imwrite(os.path.join('application_data', 'input_image', 'input_image.jpg'), frame)
        results, verified = verify(model, 0.5, 0.5)
        print(f'Verified: {verified}')
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    
cap.release()
cv2.destroyAllWindows()